In [126]:
import pymysql
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from tqdm.notebook import tqdm
import json
import os

In [127]:
with open("./envs.json","r",encoding="utf8") as f:
    envs = json.load(f)

In [128]:
es = Elasticsearch(
    hosts=envs['es_host'],
    port=envs['es_port'],
    http_auth=(envs['es_user'],envs['es_password']),
    timeout=envs['es_timeout'],
    max_retries=envs['es_max_retries']
)

In [139]:
datatype = "ntis_rnd_paper"
query = {
    "query": {
        "bool": {
            "must": [
                {"match": {"DataType": datatype}}
            ],
            "filter": [
                {
                    "range": {
                        "SearchDate": {"gte": "2023-01-01", "lte": "2023-09-30", "format": "yyyy-MM-dd"}
                    }
                }
            ]
        }
    }
}

In [140]:
search_results = es.search(
    index="source_data",
    body=query,
    track_total_hits=True,
    size=100000  # 조정 가능한 결과 크기 (최대 10만건)
)

/tmp/ipykernel_487274/682086262.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  search_results = es.search(


In [141]:
search_results

{'took': 230,
 'timed_out': False,
 '_shards': {'total': 6, 'successful': 6, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5940, 'relation': 'eq'},
  'max_score': 6.11984,
  'hits': [{'_index': 'source_data',
    '_type': '_doc',
    '_id': 'h7YWWocBhzuxVGWK99_l',
    '_score': 6.11984,
    '_source': {'BusinessNum': '1040720059',
     'DataType': 'ntis_rnd_paper',
     'SearchDate': '2023-04-07 01:33:28.697921',
     'SearchID': 'autoSystem',
     'Data': None}},
   {'_index': 'source_data',
    '_type': '_doc',
    '_id': 'f7YWWocBhzuxVGWK99-a',
    '_score': 6.11984,
    '_source': {'BusinessNum': '1040671212',
     'DataType': 'ntis_rnd_paper',
     'SearchDate': '2023-04-07 01:32:12.218442',
     'SearchID': 'autoSystem',
     'Data': None}},
   {'_index': 'source_data',
    '_type': '_doc',
    '_id': 'frYWWocBhzuxVGWK99-O',
    '_score': 6.11984,
    '_source': {'BusinessNum': '1040657165',
     'DataType': 'ntis_rnd_paper',
     'SearchDate': '2023-04-07 01:32:09.323

In [142]:
# 데이터 분할하여 리스트에 추가하기
scroll_size = 10000  # 한 번에 가져올 문서 수
scroll = scan(client=es, query=query, index="source_data", scroll='5m', size=scroll_size)

data = []
for result in tqdm(scroll, total=search_results['hits']['total']['value'] // scroll_size):
    data.append(result['_source'])

0it [00:00, ?it/s]

In [148]:
print(f"{datatype} 데이터 개수 : ", len(data))

ntis_rnd_paper 데이터 개수 :  5940


In [144]:
folder_path = "./elastic_backup_json"  # 폴더 경로 지정

# 폴더가 존재하지 않으면 폴더 생성
if not os.path.exists(folder_path):
    try:
        os.makedirs(folder_path)
        print(f"폴더 '{folder_path}'가 생성되었습니다.")
    except OSError as e:
        print(f"폴더 생성 실패: {e}")
else:
    print(f"폴더 '{folder_path}'는 이미 존재합니다.")


폴더 './elastic_backup_json'는 이미 존재합니다.


In [145]:
# 검색 결과를 JSON 파일로 저장
with open(f'{folder_path}/{datatype}.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [146]:
import os
import json

folder_path = "./elastic_backup_json"  # 폴더 경로 지정

# 폴더 내의 파일들을 검색하고 JSON 파일인 경우 데이터 개수와 파일 크기 출력
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                data_count = len(data) if isinstance(data, list) else 1
                file_size = os.path.getsize(file_path) / (1024 * 1024)  # 파일 크기를 MB 단위로 변환
                print(f"파일 '{file_name}': 데이터 개수 - {data_count}, 파일 크기 - {file_size:.2f} MB")
            except json.JSONDecodeError as e:
                print(f"파일 '{file_name}'은 JSON 형식이 아닙니다.")
else:
    print(f"폴더 '{folder_path}'를 찾을 수 없습니다.")


파일 'jobplanet_premium.json': 데이터 개수 - 501, 파일 크기 - 0.49 MB
파일 'jobplanet_review.json': 데이터 개수 - 21, 파일 크기 - 0.00 MB
파일 'jobplanet_statistic.json': 데이터 개수 - 29, 파일 크기 - 0.01 MB
파일 'kci.json': 데이터 개수 - 60000, 파일 크기 - 210.99 MB
파일 'kisti_article.json': 데이터 개수 - 3524, 파일 크기 - 7.60 MB
파일 'kisti_patent.json': 데이터 개수 - 60000, 파일 크기 - 116.14 MB
파일 'naver_blog.json': 데이터 개수 - 60000, 파일 크기 - 47.52 MB
파일 'naver_cafe.json': 데이터 개수 - 60000, 파일 크기 - 36.39 MB
파일 'naver_news.json': 데이터 개수 - 65, 파일 크기 - 0.17 MB
파일 'naver_trend.json': 데이터 개수 - 24141, 파일 크기 - 9.98 MB
파일 'ntis_accomp.json': 데이터 개수 - 4169, 파일 크기 - 0.95 MB
파일 'ntis_assign.json': 데이터 개수 - 10952, 파일 크기 - 27.27 MB
파일 'ntis_org_info.json': 데이터 개수 - 2772, 파일 크기 - 0.63 MB
파일 'ntis_rnd_paper.json': 데이터 개수 - 5940, 파일 크기 - 2.27 MB
